**LSTM models for Human Activity Recognition**

Experiements on [Pamap2](https://archive.ics.uci.edu/ml/datasets/pamap2+physical+activity+monitoring) dataset using different combinations of 
*with/without* x *temporal and/or spatial attention* x *1 or 2 LSTM layer(s)*



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pyyaml h5py 
!pip install -q tensorflow-addons
!pip install keras
!pip install pyts


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 4.9 MB/s 


In [5]:
# https://github.com/Debarshi-Bhattacharya/Ensem_HAR/blob/9d7769f34258185c56feb7c34f6059e07469030f/Implementation_on_PAMAP2/datapreprocessing.ipynb

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import math
import h5py
from sklearn.preprocessing import MinMaxScaler

'''
0: 'transient', 1: 'lying', 2: 'sitting', 3: 'standing', 4: 'walking', 5: 'running', 6: 'cycling', 7: 'Nordic_walking', 9: 'watching_TV', 
10: 'computer_work', 11: 'car driving', 12: 'ascending_stairs', 13: 'descending_stairs', 16: 'vacuum_cleaning', 17: 'ironing', 
18: 'folding_laundry', 19: 'house_cleaning', 20: 'playing_soccer', 24: 'rope_jumping'
'''

def read_files():
    list_of_files = ['Protocol/subject101.dat',
                     'Protocol/subject102.dat',
                     'Protocol/subject103.dat',
                     'Protocol/subject104.dat',
                     'Protocol/subject105.dat',
                     'Protocol/subject106.dat',
                     'Protocol/subject107.dat',
                     'Protocol/subject108.dat',
                     'Protocol/subject109.dat']
    
    subjectID = [1,2,3,4,5,6,7,8,9]
    
    # there are 54 columns in the data files
    colNames = ["timestamp", "activityID","heartrate"] # 1, 2, 3
    IMUhand = ['handTemperature', 
               'handAcc16_1', 'handAcc16_2', 'handAcc16_3', 
               'handAcc6_1', 'handAcc6_2', 'handAcc6_3', 
               'handGyro1', 'handGyro2', 'handGyro3', 
               'handMagne1', 'handMagne2', 'handMagne3',
               'handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4'] # 4-20
    IMUchest = ['chestTemperature', 
               'chestAcc16_1', 'chestAcc16_2', 'chestAcc16_3', 
               'chestAcc6_1', 'chestAcc6_2', 'chestAcc6_3', 
               'chestGyro1', 'chestGyro2', 'chestGyro3', 
               'chestMagne1', 'chestMagne2', 'chestMagne3',
               'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4'] # 21-37
    IMUankle = ['ankleTemperature', 
               'ankleAcc16_1', 'ankleAcc16_2', 'ankleAcc16_3', 
               'ankleAcc6_1', 'ankleAcc6_2', 'ankleAcc6_3', 
               'ankleGyro1', 'ankleGyro2', 'ankleGyro3', 
               'ankleMagne1', 'ankleMagne2', 'ankleMagne3',
               'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4'] # 38-54
    
    columns = colNames + IMUhand + IMUchest + IMUankle
    
    dataCollection = pd.DataFrame()

    for file in list_of_files:
        print(file)
        procData = pd.read_table(file, header=None, sep='\s+')
        procData.columns = columns
        procData['subject_id'] = int(file[-5])
        dataCollection = dataCollection.append(procData, ignore_index=True) 
        
    dataCollection.reset_index(drop=True, inplace=True)
    
    return dataCollection

data = read_files()
data.head()


Protocol/subject101.dat
Protocol/subject102.dat
Protocol/subject103.dat
Protocol/subject104.dat
Protocol/subject105.dat
Protocol/subject106.dat
Protocol/subject107.dat
Protocol/subject108.dat
Protocol/subject109.dat


,timestamp,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,...,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,ankleOrientation1,ankleOrientation2,ankleOrientation3,ankleOrientation4,subject_id
0,8.38,0,104.0,30.0,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.009250,-0.017580,-61.1888,-38.9599,-58.1438,1.0,0.0,0.0,0.0,1
1,8.39,0,NaN,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.004638,0.000368,-59.8479,-38.8919,-58.5253,1.0,0.0,0.0,0.0,1
2,8.40,0,NaN,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.000148,0.022495,-60.7361,-39.4138,-58.3999,1.0,0.0,0.0,0.0,1
3,8.41,0,NaN,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,-0.020301,0.011275,-60.4091,-38.7635,-58.3956,1.0,0.0,0.0,0.0,1
4,8.42,0,NaN,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694,1.0,0.0,0.0,0.0,1


In [6]:

def dataCleaning(dataCollection):
    dataCollection = dataCollection.drop(['timestamp', 'handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4',
                                         'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4',
                                         'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4'],
                                         axis = 1)  # removal of orientation columns as they are not needed
    dataCollection = dataCollection.drop(dataCollection[dataCollection.activityID == 0].index) # removal of any row of activity 0 as it is transient activity which it is not used
    dataCollection = dataCollection.apply(pd.to_numeric, errors = 'coerce') # removal of non numeric data in cells
    dataCollection = dataCollection.drop(['heartrate'], axis = 1)
    dataCollection = dataCollection.dropna()

    dataCollection = dataCollection.drop(['handTemperature', 'chestTemperature', 'ankleTemperature'],
                                         axis = 1)  # removal of temperature columns as they are not needed - sumeyye
    print("data cleaned!")
    return dataCollection

cleaned_data = dataCleaning(data)
print(cleaned_data['activityID'].value_counts())
cleaned_data.head(10)



data cleaned!
17    237902
4     229709
1     192290
3     188984
2     184645
7     184444
16    174976
6     163302
12    117094
13    104865
5      95641
24     47579
Name: activityID, dtype: int64


,activityID,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,handGyro2,handGyro3,...,ankleAcc6_1,ankleAcc6_2,ankleAcc6_3,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,subject_id
2928,1,2.21530,8.27915,5.58753,2.24689,8.55387,5.77143,-0.004750,0.037579,-0.011145,...,9.63162,-1.76757,0.265761,0.002908,-0.027714,0.001752,-61.1081,-36.8636,-58.3696,1
2929,1,2.29196,7.67288,5.74467,2.27373,8.14592,5.78739,-0.171710,0.025479,-0.009538,...,9.58649,-1.75247,0.250816,0.020882,0.000945,0.006007,-60.8916,-36.3197,-58.3656,1
2930,1,2.29090,7.14240,5.82342,2.26966,7.66268,5.78846,-0.238241,0.011214,0.000831,...,9.60196,-1.73721,0.356632,-0.035392,-0.052422,-0.004882,-60.3407,-35.7842,-58.6119,1
2931,1,2.21800,7.14365,5.89930,2.22177,7.25535,5.88000,-0.192912,0.019053,0.013374,...,9.58674,-1.78264,0.311453,-0.032514,-0.018844,0.026950,-60.7646,-37.1028,-57.8799,1
2932,1,2.30106,7.25857,6.09259,2.20720,7.24042,5.95555,-0.069961,-0.018328,0.004582,...,9.64677,-1.75240,0.295902,0.001351,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,1
2933,1,2.07165,7.25965,6.01218,2.19238,7.21038,6.01604,0.063895,0.007175,0.024701,...,9.60177,-1.75239,0.311276,0.003793,-0.026906,0.004125,-61.3257,-36.9744,-57.7501,1
2934,1,2.41148,7.59780,5.93915,2.23988,7.46679,6.03053,0.190837,0.003116,0.038762,...,9.67694,-1.76748,0.326060,0.036814,-0.032277,-0.006866,-61.5520,-36.9632,-57.9957,1
2935,1,2.32815,7.63431,5.70686,2.31663,7.64745,6.01495,0.200328,-0.009266,0.068567,...,9.61685,-1.76749,0.326380,-0.010352,-0.016621,0.006548,-61.5738,-36.1724,-59.3487,1
2936,1,2.25096,7.78598,5.62821,2.28637,7.70801,5.93935,0.204098,-0.068256,0.050000,...,9.61686,-1.72212,0.326234,0.039346,0.020393,-0.011880,-61.7741,-37.1744,-58.1199,1
2937,1,2.14107,7.52262,5.78141,2.31538,7.72276,5.78828,0.171291,-0.055411,0.021576,...,9.63189,-1.70699,0.326105,0.029874,-0.010763,0.005133,-60.7680,-37.4206,-58.8735,1


In [7]:
def reset_label(dataCollection): 
    # Convert original labels {1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24} to new labels. 
    mapping = {24:0,1:1,2:2,3:3,4:4,5:5,6:6,7:7,12:8,13:9,16:10,17:11} # old activity Id to new activity Id 
    for i in [24,12,13,16,17]:
        dataCollection.loc[dataCollection.activityID == i, 'activityID'] = mapping[i]

    return dataCollection
data_reset = reset_label(cleaned_data)  
data_reset.head(10)

,activityID,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,handGyro2,handGyro3,...,ankleAcc6_1,ankleAcc6_2,ankleAcc6_3,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,subject_id
2928,1,2.21530,8.27915,5.58753,2.24689,8.55387,5.77143,-0.004750,0.037579,-0.011145,...,9.63162,-1.76757,0.265761,0.002908,-0.027714,0.001752,-61.1081,-36.8636,-58.3696,1
2929,1,2.29196,7.67288,5.74467,2.27373,8.14592,5.78739,-0.171710,0.025479,-0.009538,...,9.58649,-1.75247,0.250816,0.020882,0.000945,0.006007,-60.8916,-36.3197,-58.3656,1
2930,1,2.29090,7.14240,5.82342,2.26966,7.66268,5.78846,-0.238241,0.011214,0.000831,...,9.60196,-1.73721,0.356632,-0.035392,-0.052422,-0.004882,-60.3407,-35.7842,-58.6119,1
2931,1,2.21800,7.14365,5.89930,2.22177,7.25535,5.88000,-0.192912,0.019053,0.013374,...,9.58674,-1.78264,0.311453,-0.032514,-0.018844,0.026950,-60.7646,-37.1028,-57.8799,1
2932,1,2.30106,7.25857,6.09259,2.20720,7.24042,5.95555,-0.069961,-0.018328,0.004582,...,9.64677,-1.75240,0.295902,0.001351,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,1
2933,1,2.07165,7.25965,6.01218,2.19238,7.21038,6.01604,0.063895,0.007175,0.024701,...,9.60177,-1.75239,0.311276,0.003793,-0.026906,0.004125,-61.3257,-36.9744,-57.7501,1
2934,1,2.41148,7.59780,5.93915,2.23988,7.46679,6.03053,0.190837,0.003116,0.038762,...,9.67694,-1.76748,0.326060,0.036814,-0.032277,-0.006866,-61.5520,-36.9632,-57.9957,1
2935,1,2.32815,7.63431,5.70686,2.31663,7.64745,6.01495,0.200328,-0.009266,0.068567,...,9.61685,-1.76749,0.326380,-0.010352,-0.016621,0.006548,-61.5738,-36.1724,-59.3487,1
2936,1,2.25096,7.78598,5.62821,2.28637,7.70801,5.93935,0.204098,-0.068256,0.050000,...,9.61686,-1.72212,0.326234,0.039346,0.020393,-0.011880,-61.7741,-37.1744,-58.1199,1
2937,1,2.14107,7.52262,5.78141,2.31538,7.72276,5.78828,0.171291,-0.055411,0.021576,...,9.63189,-1.70699,0.326105,0.029874,-0.010763,0.005133,-60.7680,-37.4206,-58.8735,1


In [8]:
X=data_reset.drop(['activityID'],axis=1)
y=data_reset['activityID']

X.head()

,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,handGyro2,handGyro3,handMagne1,...,ankleAcc6_1,ankleAcc6_2,ankleAcc6_3,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,subject_id
2928,2.21530,8.27915,5.58753,2.24689,8.55387,5.77143,-0.004750,0.037579,-0.011145,8.93200,...,9.63162,-1.76757,0.265761,0.002908,-0.027714,0.001752,-61.1081,-36.8636,-58.3696,1
2929,2.29196,7.67288,5.74467,2.27373,8.14592,5.78739,-0.171710,0.025479,-0.009538,9.58300,...,9.58649,-1.75247,0.250816,0.020882,0.000945,0.006007,-60.8916,-36.3197,-58.3656,1
2930,2.29090,7.14240,5.82342,2.26966,7.66268,5.78846,-0.238241,0.011214,0.000831,9.05516,...,9.60196,-1.73721,0.356632,-0.035392,-0.052422,-0.004882,-60.3407,-35.7842,-58.6119,1
2931,2.21800,7.14365,5.89930,2.22177,7.25535,5.88000,-0.192912,0.019053,0.013374,9.92698,...,9.58674,-1.78264,0.311453,-0.032514,-0.018844,0.026950,-60.7646,-37.1028,-57.8799,1
2932,2.30106,7.25857,6.09259,2.20720,7.24042,5.95555,-0.069961,-0.018328,0.004582,9.15626,...,9.64677,-1.75240,0.295902,0.001351,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,1


In [9]:
X_subID=X['subject_id']

def scale(df): # minmax scale
    features=df.columns[0:X.shape[1]]
    scaler = MinMaxScaler(feature_range=(-1,1))
    df[features]=scaler.fit_transform(df[features])
    return df

data_scaled =scale(X)
data_scaled.shape
X_scaled=pd.concat([pd.DataFrame(y,columns = ['activityID']),pd.DataFrame(data_scaled)],axis=1)
X_scaled=pd.concat([pd.DataFrame(X_scaled),pd.DataFrame(X_subID,columns = ['subject_id'])],axis=1)

X_scaled.head(10)

,activityID,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,handGyro2,handGyro3,...,ankleAcc6_2,ankleAcc6_3,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,subject_id,subject_id
2928,1,0.417516,-0.133999,-0.174116,0.113009,0.134484,0.093285,0.031349,-0.125912,-0.003356,...,-0.029670,0.015502,0.186908,0.141361,-0.082024,-0.154691,-0.129512,-0.644683,-1.0,1
2929,1,0.418253,-0.138662,-0.172903,0.113480,0.127909,0.093543,0.025227,-0.126503,-0.003244,...,-0.029426,0.015259,0.187797,0.143168,-0.081745,-0.153053,-0.124827,-0.644651,-1.0,1
2930,1,0.418242,-0.142743,-0.172296,0.113408,0.120122,0.093560,0.022788,-0.127200,-0.002519,...,-0.029180,0.016977,0.185013,0.139803,-0.082458,-0.148886,-0.120213,-0.646624,-1.0,1
2931,1,0.417542,-0.142733,-0.171710,0.112568,0.113557,0.095039,0.024450,-0.126817,-0.001641,...,-0.029913,0.016243,0.185156,0.141920,-0.080374,-0.152093,-0.131573,-0.640759,-1.0,1
2932,1,0.418340,-0.141849,-0.170219,0.112313,0.113316,0.096259,0.028958,-0.128644,-0.002256,...,-0.029425,0.015991,0.186831,0.140026,-0.082553,-0.147852,-0.131743,-0.640798,-1.0,1
2933,1,0.416137,-0.141841,-0.170839,0.112053,0.112832,0.097235,0.033865,-0.127398,-0.000850,...,-0.029425,0.016241,0.186951,0.141412,-0.081868,-0.156337,-0.130467,-0.639719,-1.0,1
2934,1,0.419401,-0.139240,-0.171403,0.112886,0.116965,0.097469,0.038519,-0.127596,0.000134,...,-0.029668,0.016480,0.188585,0.141073,-0.082588,-0.158048,-0.130370,-0.641687,-1.0,1
2935,1,0.418600,-0.138959,-0.173195,0.114232,0.119876,0.097218,0.038867,-0.128201,0.002218,...,-0.029669,0.016486,0.186252,0.142060,-0.081710,-0.158213,-0.123558,-0.652528,-1.0,1
2936,1,0.417859,-0.137792,-0.173802,0.113701,0.120852,0.095997,0.039005,-0.131084,0.000920,...,-0.028937,0.016483,0.188710,0.144395,-0.082916,-0.159728,-0.132190,-0.642682,-1.0,1
2937,1,0.416803,-0.139818,-0.172620,0.114210,0.121090,0.093558,0.037803,-0.130456,-0.001068,...,-0.028692,0.016481,0.188241,0.142430,-0.081802,-0.152118,-0.134311,-0.648720,-1.0,1


In [10]:
SLIDING_WINDOW_LENGTH = 100

def segment_signal(data, window_size): # data is numpy array
    n = len(data)
    X, y = [], []
    start, end = 0, 0
    while start + window_size - 1 < n:
        end = start + window_size-1
        # if the frame contains the same activity and from the same object
        X.append(data[start:(end+1),1:-1])
        y.append(data[start][0])
        start += window_size #without overlap (for 50% overlap use window_size//2)
    print(np.asarray(X).shape, np.asarray(y).shape)
    return {'inputs' : np.asarray(X), 'labels': np.asarray(y,dtype=int)}

data_segmented=segment_signal(X_scaled.to_numpy(),SLIDING_WINDOW_LENGTH)

(19214, 100, 37) (19214,)


In [11]:
def save_data(data,file_name): # save the data in h5 format
    f = h5py.File(file_name,'w')
    for key in data:
        f.create_dataset(key,data = data[key])       
    f.close()   

file_name = 'pamap_scaled_segmented_100.h5'

save_data(data_segmented, file_name)
print("File is saved.")

File is saved.


In [12]:
from keras.models import Model
from tensorflow.keras.utils import to_categorical

path = "pamap_scaled_segmented_100.h5"

f = h5py.File(path, 'r')

data_x = np.array(f["inputs"][:]) 
data_y = np.array(f["labels"][:])

print(data_x.shape)
print(data_y.shape)

(19214, 100, 37)
(19214,)


In [13]:
# https://stackoverflow.com/questions/53731141/cifar10-randomize-train-and-test-set
def shuffle_train_data(X_train, Y_train): 
    """called after each epoch""" 
    perm = np.random.permutation(len(Y_train)) 
    Xtr_shuf = X_train[perm] 
    Ytr_shuf = Y_train[perm] 
    return Xtr_shuf, Ytr_shuf 
X_shuffled, y_shuffled = shuffle_train_data(data_x, data_y) 
print(X_shuffled.shape) 
print(y_shuffled.shape)


(19214, 100, 37)
(19214,)


In [14]:
#https://stackoverflow.com/questions/53731141/cifar10-randomize-train-and-test-set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, test_size=0.33, random_state=1234)
# Check shape
print(X_train.shape) 
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(12873, 100, 37)
(12873,)
(6341, 100, 37)
(6341,)


In [20]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Flatten, Input
from keras import optimizers, losses, metrics, initializers

from collections import Counter
NUM_CLASSES = len(Counter(y_shuffled).keys()) # Hardcoded number of classes in the gesture recognition problem
BATCH_SIZE = 50 # Batch Size
NUM_UNITS_LSTM = 16 # Number of unit in the long short-term recurrent layers
NB_SENSOR_CHANNELS = data_x.shape[2]
SLIDING_WINDOW_LENGTH = data_x.shape[1]

In [22]:
model = Sequential()
model._name="Experiement1_1LSTM_without_Attention"
model.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model.add(Flatten())

#Applying a dense layer of softmax.
model.add(Dense(NUM_CLASSES))

model.summary()

model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2)
model.evaluate(X_test, y_test)

Model: "Experiement1_1LSTM_without_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100, 16)           3456      
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 12)                19212     
                                                                 
Total params: 22,668
Trainable params: 22,668
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/206 [==============================] - 11s 13ms/step - loss: 1.4186 - sparse_categorical_accuracy: 0.5538 - val_loss: 1.1230 - val_sparse_categorical_accuracy: 0.6342
Epoch 2/30
206/206 [==============================] - 2s 10ms/step - loss: 0.9893 - sparse_categorical_accur

[0.37246766686439514, 0.8959154486656189]

In [23]:
# https://github.com/ManzhuYu/Code-SpatioTemporalAttention-LSTM-main/blob/main/modelbase.py

model_2 = Sequential()
model_2._name="Experiement2_1LSTM_with_temporal_Attention"
model_2.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_2.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_2.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module
model_2.add(Flatten())

#Applying a dense layer of softmax.
model_2.add(Dense(NUM_CLASSES))

model_2.summary()

model_2.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_2.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_2.evaluate(X_test, y_test)

Model: "Experiement2_1LSTM_with_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100, 16)           3456      
                                                                 
 dense_3 (Dense)             (None, 100, 1600)         27200     
                                                                 
 flatten_3 (Flatten)         (None, 160000)            0         
                                                                 
 dense_4 (Dense)             (None, 12)                1920012   
                                                                 
Total params: 1,950,668
Trainable params: 1,950,668
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/206 [==============================] - 4s 13ms/step - loss: 1.3131 - sparse_categorical_accuracy: 0.5967 - val_loss: 1.0788 - val_s

[0.3592246472835541, 0.9064816236495972]

In [24]:
model_3 = Sequential()
model_3._name="Experiement3_1LSTM_with_spatial_Attention"
model_3.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_3.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_3.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_3.add(Flatten())

#Applying a dense layer of softmax.
model_3.add(Dense(NUM_CLASSES))

model_3.summary()

model_3.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_3.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_3.evaluate(X_test, y_test)

Model: "Experiement3_1LSTM_with_spatial_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 100, 100)          3800      
                                                                 
 lstm_4 (LSTM)               (None, 100, 16)           7488      
                                                                 
 flatten_4 (Flatten)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 12)                19212     
                                                                 
Total params: 30,500
Trainable params: 30,500
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/206 [==============================] - 4s 11ms/step - loss: 1.2637 - sparse_categorical_accuracy: 0.6079 - val_loss: 0.9515 - val_sparse_c

[0.28276267647743225, 0.9307680130004883]

In [25]:
model_4 = Sequential()
model_4._name="Experiement4_1LSTM_with_spatial_temporal_Attention"
model_4.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_4.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_4.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_4.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module
model_4.add(Flatten())

#Applying a dense layer of softmax.
model_4.add(Dense(NUM_CLASSES))

model_4.summary()

model_4.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_4.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_4.evaluate(X_test, y_test)

Model: "Experiement4_1LSTM_with_spatial_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100, 100)          3800      
                                                                 
 lstm_5 (LSTM)               (None, 100, 16)           7488      
                                                                 
 dense_8 (Dense)             (None, 100, 1600)         27200     
                                                                 
 flatten_5 (Flatten)         (None, 160000)            0         
                                                                 
 dense_9 (Dense)             (None, 12)                1920012   
                                                                 
Total params: 1,958,500
Trainable params: 1,958,500
Non-trainable params: 0
_________________________________________________________________
Epoch 

[0.3405660092830658, 0.9159438610076904]

In [26]:
model_5a = Sequential()
model_5a._name="Experiement5a_2LSTM_with_temporal_Attention"
model_5a.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_5a.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_5a.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module
model_5a.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_5a.add(Flatten())

#Applying a dense layer of softmax.
model_5a.add(Dense(NUM_CLASSES))

model_5a.summary()

model_5a.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_5a.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_5a.evaluate(X_test, y_test)

Model: "Experiement5a_2LSTM_with_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 100, 16)           3456      
                                                                 
 dense_10 (Dense)            (None, 100, 1600)         27200     
                                                                 
 lstm_7 (LSTM)               (None, 100, 16)           103488    
                                                                 
 flatten_6 (Flatten)         (None, 1600)              0         
                                                                 
 dense_11 (Dense)            (None, 12)                19212     
                                                                 
Total params: 153,356
Trainable params: 153,356
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/20

[0.26835426688194275, 0.9307680130004883]

In [27]:
model_5b = Sequential()
model_5b._name="Experiement5b_2LSTM_with_temporal_Attention"
model_5b.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_5b.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_5b.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_5b.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module
model_5b.add(Flatten())

#Applying a dense layer of softmax.
model_5b.add(Dense(NUM_CLASSES))

model_5b.summary()

model_5b.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_5b.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_5b.evaluate(X_test, y_test)

Model: "Experiement5b_2LSTM_with_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100, 16)           3456      
                                                                 
 lstm_9 (LSTM)               (None, 100, 16)           2112      
                                                                 
 dense_12 (Dense)            (None, 100, 1600)         27200     
                                                                 
 flatten_7 (Flatten)         (None, 160000)            0         
                                                                 
 dense_13 (Dense)            (None, 12)                1920012   
                                                                 
Total params: 1,952,780
Trainable params: 1,952,780
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
20

[0.36121025681495667, 0.9028544425964355]

In [28]:
model_5c = Sequential()
model_5c._name="Experiement5c_2LSTM_with_temporal_Attention"
model_5c.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_5c.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module
model_5c.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_5c.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_5c.add(Flatten())

#Applying a dense layer of softmax.
model_5c.add(Dense(NUM_CLASSES))

model_5c.summary()

model_5c.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_5c.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_5c.evaluate(X_test, y_test)

Model: "Experiement5c_2LSTM_with_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 100, 1600)         60800     
                                                                 
 lstm_10 (LSTM)              (None, 100, 16)           103488    
                                                                 
 lstm_11 (LSTM)              (None, 100, 16)           2112      
                                                                 
 flatten_8 (Flatten)         (None, 1600)              0         
                                                                 
 dense_15 (Dense)            (None, 12)                19212     
                                                                 
Total params: 185,612
Trainable params: 185,612
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/20

[0.29685431718826294, 0.9110550284385681]

In [29]:
model_6a = Sequential()
model_6a._name="Experiement6a_2LSTM_with_spatial_Attention"
model_6a.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_6a.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_6a.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_6a.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_6a.add(Flatten())

#Applying a dense layer of softmax.
model_6a.add(Dense(NUM_CLASSES))

model_6a.summary()

model_6a.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_6a.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_6a.evaluate(X_test, y_test)

Model: "Experiement6a_2LSTM_with_spatial_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 100, 16)           3456      
                                                                 
 dense_16 (Dense)            (None, 100, 100)          1700      
                                                                 
 lstm_13 (LSTM)              (None, 100, 16)           7488      
                                                                 
 flatten_9 (Flatten)         (None, 1600)              0         
                                                                 
 dense_17 (Dense)            (None, 12)                19212     
                                                                 
Total params: 31,856
Trainable params: 31,856
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/206 [

[0.3147575259208679, 0.9190979599952698]

In [30]:
model_6b = Sequential()
model_6b._name="Experiement6b_2LSTM_with_spatial_Attention"
model_6b.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_6b.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_6b.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_6b.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_6b.add(Flatten())

#Applying a dense layer of softmax.
model_6b.add(Dense(NUM_CLASSES))

model_6b.summary()

model_6b.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_6b.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_6b.evaluate(X_test, y_test)

Model: "Experiement6b_2LSTM_with_spatial_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 100, 16)           3456      
                                                                 
 lstm_15 (LSTM)              (None, 100, 16)           2112      
                                                                 
 dense_18 (Dense)            (None, 100, 100)          1700      
                                                                 
 flatten_10 (Flatten)        (None, 10000)             0         
                                                                 
 dense_19 (Dense)            (None, 12)                120012    
                                                                 
Total params: 127,280
Trainable params: 127,280
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/206

[0.304587721824646, 0.9123166799545288]

In [31]:
model_6c = Sequential()
model_6c._name="Experiement6c_2LSTM_with_spatial_Attention"
model_6c.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_6c.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_6c.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_6c.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_6c.add(Flatten())

#Applying a dense layer of softmax.
model_6c.add(Dense(NUM_CLASSES))

model_6c.summary()

model_6c.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_6c.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_6c.evaluate(X_test, y_test)

Model: "Experiement6c_2LSTM_with_spatial_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 100, 100)          3800      
                                                                 
 lstm_16 (LSTM)              (None, 100, 16)           7488      
                                                                 
 lstm_17 (LSTM)              (None, 100, 16)           2112      
                                                                 
 flatten_11 (Flatten)        (None, 1600)              0         
                                                                 
 dense_21 (Dense)            (None, 12)                19212     
                                                                 
Total params: 32,612
Trainable params: 32,612
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/206 [

[0.3044148087501526, 0.924144446849823]

In [32]:
model_7a = Sequential()
model_7a._name="Experiement7a_2LSTM_with_spatial_temporal_Attention"
model_7a.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_7a.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_7a.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7a.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7a.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module

model_7a.add(Flatten())

#Applying a dense layer of softmax.
model_7a.add(Dense(NUM_CLASSES))

model_7a.summary()

model_7a.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_7a.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_7a.evaluate(X_test, y_test)

Model: "Experiement7a_2LSTM_with_spatial_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 100, 100)          3800      
                                                                 
 lstm_18 (LSTM)              (None, 100, 16)           7488      
                                                                 
 lstm_19 (LSTM)              (None, 100, 16)           2112      
                                                                 
 dense_23 (Dense)            (None, 100, 1600)         27200     
                                                                 
 flatten_12 (Flatten)        (None, 160000)            0         
                                                                 
 dense_24 (Dense)            (None, 12)                1920012   
                                                                 
Total params: 1

[0.30051830410957336, 0.923040509223938]

In [33]:
model_7b = Sequential()
model_7b._name="Experiement7b_2LSTM_with_spatial_temporal_Attention"
model_7b.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_7b.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7b.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_7b.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7b.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module

model_7b.add(Flatten())

#Applying a dense layer of softmax.
model_7b.add(Dense(NUM_CLASSES))

model_7b.summary()

model_7b.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_7b.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_7b.evaluate(X_test, y_test)

Model: "Experiement7b_2LSTM_with_spatial_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 100, 16)           3456      
                                                                 
 dense_25 (Dense)            (None, 100, 100)          1700      
                                                                 
 lstm_21 (LSTM)              (None, 100, 16)           7488      
                                                                 
 dense_26 (Dense)            (None, 100, 1600)         27200     
                                                                 
 flatten_13 (Flatten)        (None, 160000)            0         
                                                                 
 dense_27 (Dense)            (None, 12)                1920012   
                                                                 
Total params: 1

[0.2862699627876282, 0.9268254041671753]

In [35]:
model_7c = Sequential()
model_7c._name="Experiement7c_2LSTM_with_spatial_temporal_Attention"
model_7c.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_7c.add(Dense(SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # spatial module
model_7c.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7c.add(Dense(NUM_UNITS_LSTM*SLIDING_WINDOW_LENGTH, input_shape=(SLIDING_WINDOW_LENGTH,), activation=None)) # temporal module
model_7c.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7c.add(Flatten())

#Applying a dense layer of softmax.
model_7c.add(Dense(NUM_CLASSES))

model_7c.summary()

model_7c.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_7c.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_7c.evaluate(X_test, y_test)

Model: "Experiement7c_2LSTM_with_spatial_temporal_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 100, 100)          3800      
                                                                 
 lstm_24 (LSTM)              (None, 100, 16)           7488      
                                                                 
 dense_32 (Dense)            (None, 100, 1600)         27200     
                                                                 
 lstm_25 (LSTM)              (None, 100, 16)           103488    
                                                                 
 flatten_15 (Flatten)        (None, 1600)              0         
                                                                 
 dense_33 (Dense)            (None, 12)                19212     
                                                                 
Total params: 1

[0.28986841440200806, 0.9317142367362976]

In [36]:
model_7d = Sequential()
model_7d._name="Experiement7d_2LSTM_without_Attention"
model_7d.add(Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)))

#intializing weights
initializer = initializers.Orthogonal()
model_7d.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7d.add(LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model_7d.add(Flatten())

#Applying a dense layer of softmax.
model_7d.add(Dense(NUM_CLASSES))

model_7d.summary()

model_7d.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()],
)

model_7d.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=30,validation_split=0.2) 
model_7d.evaluate(X_test, y_test)

Model: "Experiement7d_2LSTM_without_Attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_26 (LSTM)              (None, 100, 16)           3456      
                                                                 
 lstm_27 (LSTM)              (None, 100, 16)           2112      
                                                                 
 flatten_16 (Flatten)        (None, 1600)              0         
                                                                 
 dense_34 (Dense)            (None, 12)                19212     
                                                                 
Total params: 24,780
Trainable params: 24,780
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
206/206 [==============================] - 6s 17ms/step - loss: 1.4992 - sparse_categorical_accuracy: 0.5020 - val_loss: 1.2027 - val_sparse_categ

[0.33398544788360596, 0.9079009890556335]

Summary:

-   Experiement1_**1LSTM_without**_Attention: 0.8959154486656189
-   Experiement2_1LSTM_with_temporal_Attention: 0.9064816236495972
-   Experiement3_1LSTM_with_spatial_Attention: 0.9307680130004883
-   Experiement4_1LSTM_with_spatial_temporal_Attention: 0.9159438610076904
-   Experiement5a_2LSTM_with_temporal_Attention: 0.9307680130004883
-   Experiement5b_2LSTM_with_temporal_Attention: 0.9028544425964355
-   Experiement5c_2LSTM_with_temporal_Attention: 0.9110550284385681
-   Experiement6a_2LSTM_with_spatial_Attention: 0.9190979599952698
-   Experiement6b_2LSTM_with_spatial_Attention: 0.9123166799545288
-   Experiement6c_2LSTM_with_spatial_Attention: 0.924144446849823
-   Experiement7a_2LSTM_with_spatial_temporal_Attention: 0.923040509223938
-   Experiement7b_2LSTM_with_spatial_temporal_Attention: 0.9268254041671753
-   Experiement7c_2LSTM_with_spatial_temporal_Attention: 0.9317142367362976
-   Experiement7d_**2LSTM_without**_Attention: 0.9079009890556335


